In [1]:
import os
import pandas as pd
from pathlib import Path
import requests
from datetime import datetime
from io import StringIO
from dotenv import load_dotenv

load_dotenv()  # carga .env si existe

def fetch_and_save_EA(year: int, month: int, periodicity: str,
                            base_url: str = "https://billing.izoswap.fr/admin/export/suivi_error_acquisition_attempts",
                            token: str | None = None,
                            out_dir: Path = Path("..") / "data" / "Error_Adquisition") -> Path:
    """
    periodicity: 'M' (mensual) -> usa endpoint 'all/.../men,first'
                 'Q' (trimestral) -> usa endpoint 'ALL/.../first,trim'
    El token se toma de ENV PAYMENTS_TOKEN si no se pasa por parámetro.
    """
    periodicity = periodicity.upper().strip()
    if periodicity not in {"M", "Q"}:
        raise ValueError("periodicity debe ser 'M' o 'Q'")

    # Token desde env si no se pasa
    token = token or os.getenv("PAYMENTS_TOKEN")
    if not token:
        raise RuntimeError("Falta el token. Define PAYMENTS_TOKEN en el entorno o pásalo como argumento.")

    month_str = f"{month:02d}"
    ym = f"{year}-{month_str}"

    # Construcción de URL según periodicity
    if periodicity == "M":
        url = f"{base_url}/all/{ym}/{token}/men,uad"
    else:
        url = f"{base_url}/ALL/{ym}/{token}/trim,uad"

    headers = {"User-Agent": "Mozilla/5.0"}
    r = requests.get(url, headers=headers, timeout=60)
    r.raise_for_status()
    html = r.text

    # Parseo robusto
    try:
        dfs = pd.read_html(StringIO(html))  # lxml
    except Exception:
        dfs = pd.read_html(StringIO(html), flavor="html5lib")  # fallback
    if not dfs:
        raise ValueError("No se encontraron tablas en la página.")

    df = max(dfs, key=lambda d: (d.shape[1], d.shape[0])).copy()

    out_dir.mkdir(parents=True, exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"ALL_PA_{periodicity}_2025-08_{timestamp}.csv"  # según tu formato fijo
    out_path = out_dir / filename
    df.to_csv(out_path, index=False, encoding="utf-8")
    print(f"[OK] Guardado: {out_path} | shape={df.shape}")
    return out_path

# Uso:
fetch_and_save_EA(2025, 8, "M")  # lee PAYMENTS_TOKEN del entorno
fetch_and_save_EA(2025, 8, "Q")  # lee PAYMENTS_TOKEN del entorno

[OK] Guardado: ..\data\Error_Adquisition\ALL_PA_M_2025-08_20250819_184908.csv | shape=(1468, 25)
[OK] Guardado: ..\data\Error_Adquisition\ALL_PA_Q_2025-08_20250819_184914.csv | shape=(2906, 25)


WindowsPath('../data/Error_Adquisition/ALL_PA_Q_2025-08_20250819_184914.csv')